# Context

**The file we chose to use is the music by Alicia keys new york.**

**We will use Recurring Neural Networks to generate texts.**

**The Neural Network will learn from the texts so that it can write its own texts.**

**The goal is to train the LSTM to predict the next character in a text string.**


In [1]:
import tensorflow as tf
import numpy as np
import os
import time
import pandas as pd

# Database loading and exploration

In [2]:
data = '../input/poetry/alicia-keys.txt'

In [3]:
dataset_text = open(data, 'rb').read().decode(encoding = 'utf-8')

In [4]:
print(dataset_text)

Ooh....... New York x2 Grew up in a town that is famous as a place of movie scenes
Noise is always loud, there are sirens all around and the streets are mean
If I can make it here, I can make it anywhere, that's what they say
Seeing my face in lights or my name on marquees found down on Broadway Even if it ain't all it seems, I got a pocket full of dreams
Baby, I'm from New York
Concrete jungle where dreams are made of
There's nothing you can't do
Now you're in New York
These streets will make you feel brand new
Big lights will inspire you
Hear it from New York, New York, New York! On the avenue, there ain't never a curfew, ladies work so hard
Such a melting pot, on the corner selling rock, preachers pray to God
Hail a gypsy-cab, takes me down from Harlem to the Brooklyn Bridge
Some will sleep tonight with a hunger far more than an empty fridge I'm gonna make it by any means, I got a pocket full of dreams
Baby, I'm from New York
Concrete jungle where dreams are made of
There's nothing 

In [5]:
len(dataset_text)

113439

In [6]:
vocab = sorted(set(dataset_text))

In [7]:
print('{} unique characters'.format(len(vocab)))

76 unique characters


In [8]:
vocab

['\n',
 ' ',
 '!',
 '"',
 '&',
 "'",
 '(',
 ')',
 ',',
 '-',
 '.',
 '0',
 '1',
 '2',
 '4',
 '6',
 '8',
 '9',
 ':',
 '?',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'Y',
 '[',
 ']',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 '¢',
 'Ã',
 'é',
 'ó',
 'ú']

# Mapping text to numbers

In [9]:
char2idx = {char: index for index, char in enumerate(vocab)}

In [10]:
char2idx

{'\n': 0,
 ' ': 1,
 '!': 2,
 '"': 3,
 '&': 4,
 "'": 5,
 '(': 6,
 ')': 7,
 ',': 8,
 '-': 9,
 '.': 10,
 '0': 11,
 '1': 12,
 '2': 13,
 '4': 14,
 '6': 15,
 '8': 16,
 '9': 17,
 ':': 18,
 '?': 19,
 'A': 20,
 'B': 21,
 'C': 22,
 'D': 23,
 'E': 24,
 'F': 25,
 'G': 26,
 'H': 27,
 'I': 28,
 'J': 29,
 'K': 30,
 'L': 31,
 'M': 32,
 'N': 33,
 'O': 34,
 'P': 35,
 'R': 36,
 'S': 37,
 'T': 38,
 'U': 39,
 'V': 40,
 'W': 41,
 'Y': 42,
 '[': 43,
 ']': 44,
 'a': 45,
 'b': 46,
 'c': 47,
 'd': 48,
 'e': 49,
 'f': 50,
 'g': 51,
 'h': 52,
 'i': 53,
 'j': 54,
 'k': 55,
 'l': 56,
 'm': 57,
 'n': 58,
 'o': 59,
 'p': 60,
 'q': 61,
 'r': 62,
 's': 63,
 't': 64,
 'u': 65,
 'v': 66,
 'w': 67,
 'x': 68,
 'y': 69,
 'z': 70,
 '¢': 71,
 'Ã': 72,
 'é': 73,
 'ó': 74,
 'ú': 75}

In [11]:
idx2char = np.array(vocab)

In [12]:
idx2char

array(['\n', ' ', '!', '"', '&', "'", '(', ')', ',', '-', '.', '0', '1',
       '2', '4', '6', '8', '9', ':', '?', 'A', 'B', 'C', 'D', 'E', 'F',
       'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'R', 'S', 'T',
       'U', 'V', 'W', 'Y', '[', ']', 'a', 'b', 'c', 'd', 'e', 'f', 'g',
       'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't',
       'u', 'v', 'w', 'x', 'y', 'z', '¢', 'Ã', 'é', 'ó', 'ú'], dtype='<U1')

In [13]:
idx2char[10]

'.'

In [14]:
char2idx[':']

18

In [15]:
text_as_int = np.array([char2idx[char] for char in dataset_text])

In [16]:
text_as_int

array([34, 59, 52, ..., 64, 63,  1])

In [17]:
text_as_int.shape

(113439,)

In [18]:
print('{} characters mapped to int ---> {}'.format(repr(dataset_text[:13]), text_as_int[:13]))

'Ooh....... Ne' characters mapped to int ---> [34 59 52 10 10 10 10 10 10 10  1 33 49]


# Creation of training examples and batches

In [19]:
len(dataset_text)

113439

In [20]:
seq_length = 100
examples_per_epoch = len(dataset_text) // seq_length
examples_per_epoch

1134

In [21]:
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

2021-11-27 20:25:49.748461: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-27 20:25:49.857161: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-27 20:25:49.857850: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-27 20:25:49.860143: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

In [22]:
char_dataset

<TensorSliceDataset shapes: (), types: tf.int64>

In [23]:
sequences = char_dataset.batch(seq_length + 1, drop_remainder=True)

In [24]:
sequences

<BatchDataset shapes: (101,), types: tf.int64>

In [25]:
for item in sequences.take(50):
  print(repr(''.join(idx2char[item.numpy()])))

'Ooh....... New York x2 Grew up in a town that is famous as a place of movie scenes\nNoise is always lo'
'ud, there are sirens all around and the streets are mean\nIf I can make it here, I can make it anywher'
"e, that's what they say\nSeeing my face in lights or my name on marquees found down on Broadway Even i"
"f it ain't all it seems, I got a pocket full of dreams\nBaby, I'm from New York\nConcrete jungle where "
"dreams are made of\nThere's nothing you can't do\nNow you're in New York\nThese streets will make you fe"
'el brand new\nBig lights will inspire you\nHear it from New York, New York, New York! On the avenue, th'
"ere ain't never a curfew, ladies work so hard\nSuch a melting pot, on the corner selling rock, preache"
'rs pray to God\nHail a gypsy-cab, takes me down from Harlem to the Brooklyn Bridge\nSome will sleep ton'
"ight with a hunger far more than an empty fridge I'm gonna make it by any means, I got a pocket full "
"of dreams\nBaby, I'm from New York\nConcrete jungl

In [26]:
def split_input_target(chunk):
  input_text = chunk[:-1]
  target_text = chunk[1:]
  return input_text, target_text

In [27]:
dataset = sequences.map(split_input_target)

In [28]:
for input_example, target_example in dataset.take(10):
  print('Input data:', repr(''.join(idx2char[input_example.numpy()])))
  print('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data: 'Ooh....... New York x2 Grew up in a town that is famous as a place of movie scenes\nNoise is always l'
Target data: 'oh....... New York x2 Grew up in a town that is famous as a place of movie scenes\nNoise is always lo'
Input data: 'ud, there are sirens all around and the streets are mean\nIf I can make it here, I can make it anywhe'
Target data: 'd, there are sirens all around and the streets are mean\nIf I can make it here, I can make it anywher'
Input data: "e, that's what they say\nSeeing my face in lights or my name on marquees found down on Broadway Even "
Target data: ", that's what they say\nSeeing my face in lights or my name on marquees found down on Broadway Even i"
Input data: "f it ain't all it seems, I got a pocket full of dreams\nBaby, I'm from New York\nConcrete jungle where"
Target data: " it ain't all it seems, I got a pocket full of dreams\nBaby, I'm from New York\nConcrete jungle where "
Input data: "dreams are made of\nThere's nothing you can't do\nNow

2021-11-27 20:25:53.037971: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


In [29]:
batch_size = 64
buffer_size = 10000

In [30]:
dataset = dataset.shuffle(buffer_size).batch(batch_size, drop_remainder = True)

In [31]:
dataset

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

# Model building

In [32]:
len(vocab)

76

In [33]:
vocab_size = len(vocab)

In [34]:
embedding_dim = 256

In [35]:
rnn_units = 1024

In [36]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([tf.keras.layers.Embedding(vocab_size, embedding_dim, batch_input_shape=[batch_size, None]),
                               tf.keras.layers.LSTM(rnn_units, return_sequences=True, stateful=True, recurrent_initializer='glorot_uniform'),
                               tf.keras.layers.Dense(vocab_size)])
  return model

In [37]:
model = build_model(vocab_size = len(vocab), embedding_dim=embedding_dim, rnn_units=rnn_units, batch_size=batch_size)

In [38]:
for input_example_batch, target_example_batch in dataset.take(10):
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape)

2021-11-27 20:25:56.290642: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


(64, 100, 76)
(64, 100, 76)
(64, 100, 76)
(64, 100, 76)
(64, 100, 76)
(64, 100, 76)
(64, 100, 76)
(64, 100, 76)
(64, 100, 76)
(64, 100, 76)


In [39]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)

In [40]:
sampled_indices

<tf.Tensor: shape=(100, 1), dtype=int64, numpy=
array([[36],
       [60],
       [11],
       [46],
       [45],
       [67],
       [72],
       [15],
       [61],
       [41],
       [38],
       [52],
       [ 9],
       [73],
       [34],
       [71],
       [57],
       [ 0],
       [32],
       [45],
       [63],
       [26],
       [17],
       [12],
       [38],
       [44],
       [44],
       [62],
       [ 6],
       [41],
       [21],
       [38],
       [43],
       [67],
       [46],
       [71],
       [25],
       [60],
       [13],
       [72],
       [ 3],
       [47],
       [50],
       [ 4],
       [ 5],
       [34],
       [53],
       [38],
       [36],
       [10],
       [53],
       [14],
       [75],
       [11],
       [48],
       [32],
       [23],
       [57],
       [ 3],
       [43],
       [59],
       [69],
       [25],
       [ 9],
       [51],
       [54],
       [32],
       [49],
       [27],
       [41],
       [52],
       [ 6],
       [24],
   

In [41]:
sampled_indices = tf.squeeze(sampled_indices, axis = -1).numpy()

In [42]:
sampled_indices

array([36, 60, 11, 46, 45, 67, 72, 15, 61, 41, 38, 52,  9, 73, 34, 71, 57,
        0, 32, 45, 63, 26, 17, 12, 38, 44, 44, 62,  6, 41, 21, 38, 43, 67,
       46, 71, 25, 60, 13, 72,  3, 47, 50,  4,  5, 34, 53, 38, 36, 10, 53,
       14, 75, 11, 48, 32, 23, 57,  3, 43, 59, 69, 25,  9, 51, 54, 32, 49,
       27, 41, 52,  6, 24, 10, 40, 55, 24, 67, 69, 40, 70, 49, 38,  0, 50,
       30, 18, 14, 38, 12, 18, 53, 10, 30, 20, 66, 42, 36, 60, 73])

In [43]:
print('Input: \n', repr(''.join(idx2char[input_example_batch[0]])))
print()
print('Next char predictions: \n', repr(''.join(idx2char[sampled_indices])))

Input: 
 'K!\nNew day\nOh, oh, A-K!\nOw! Party people say, party people say, "Aye\nIt\'s a new day, it\'s a new day"'

Next char predictions: 
 'Rp0bawÃ6qWTh-éO¢m\nMasG91T]]r(WBT[wb¢Fp2Ã"cf&\'OiTR.i4ú0dMDm"[oyF-gjMeHWh(E.VkEwyVzeT\nfK:4T1:i.KAvYRpé'


# Model training

**Optimizer and loss function**

In [44]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

In [45]:
example_batch_loss = loss(target_example_batch, example_batch_predictions)

In [46]:
example_batch_loss.numpy().mean()

4.3313684

In [47]:
model.compile(optimizer='Adam', loss=loss)

**Checkpoints**

In [48]:
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, 'ckpt_{epoch}')
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_prefix, save_weights_only=True)

# Execution of training

In [49]:
epochs = 40
history = model.fit(dataset, epochs = epochs, callbacks=[checkpoint_callback])

Epoch 1/40
17/17 [==============================] - 3s 51ms/step - loss: 3.5421
Epoch 2/40
17/17 [==============================] - 1s 49ms/step - loss: 3.1057
Epoch 3/40
17/17 [==============================] - 1s 49ms/step - loss: 2.9809
Epoch 4/40
17/17 [==============================] - 1s 48ms/step - loss: 2.8052
Epoch 5/40
17/17 [==============================] - 1s 49ms/step - loss: 2.6320
Epoch 6/40
17/17 [==============================] - 1s 49ms/step - loss: 2.4865
Epoch 7/40
17/17 [==============================] - 1s 49ms/step - loss: 2.3825
Epoch 8/40
17/17 [==============================] - 1s 49ms/step - loss: 2.3139
Epoch 9/40
17/17 [==============================] - 1s 49ms/step - loss: 2.2468
Epoch 10/40
17/17 [==============================] - 1s 50ms/step - loss: 2.1925
Epoch 11/40
17/17 [==============================] - 1s 49ms/step - loss: 2.1424
Epoch 12/40
17/17 [==============================] - 1s 48ms/step - loss: 2.0867
Epoch 13/40
17/17 [==================

# Text Generation

Restore last checkpoint

In [50]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints/ckpt_40'

In [51]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size = 1)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

In [52]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 256)            19456     
_________________________________________________________________
lstm_1 (LSTM)                (1, None, 1024)           5246976   
_________________________________________________________________
dense_1 (Dense)              (1, None, 76)             77900     
Total params: 5,344,332
Trainable params: 5,344,332
Non-trainable params: 0
_________________________________________________________________


# Prediction Loop

In [53]:
def generate_text(model, start_string):
  
  num_generate = 1000

  
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)
  
  text_generated = []
   
  temperature = 1.0

  
  for i in range(num_generate):
    
    predictions = model(input_eval)

    
    predictions = tf.squeeze(predictions, 0)
    predictions = predictions / temperature
    predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

    
    input_eval = tf.expand_dims([predicted_id], 0)

    text_generated.append(idx2char[predicted_id])
  
  return (start_string + ''.join(text_generated))

In [54]:
print(generate_text(model, start_string='New : '))

New : just now you from (Another standon'x till of me dears al alone
Like my loster
I'm gonna live you're butwer of a fies
When you real go a velu
AKe you will I be crazy for me in too) no many realire
Time you will me you, so the tay got world The isund you) 'cause I don't wanna needs, op ey each, want to toll me, baby soophore
And the world sake our with me, say you need me, you many 've's a throp a samasqure go
How walk no mone) (how come holy wants 'Cause I've mean then pursing the dame
Some only want sten pis away
Oh me, she's that the millown, Mindat's felling down mi houch
Who try the gothing, be sure
To Make it my hoad you
Flling me if phone that you say good or tile I'm gonna till never seart when I'll rised, you can't do, what he do Whowe hep you down that I hadd I was gonea home ahay
It's tee a real your for?
Why hur holving fatter
For a ficelust Everything's gaid a lonna Just speees, babe (Might)
Dbeen there be always up love usparm (Live fire that pouse fal
That's how drem

# **If you find this notebook useful, support with an upvote** 👍